In [1]:
import pygame 
import numpy as np 
import sys 

pygame 2.5.2 (SDL 2.28.3, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
bg = pygame.image.load('back.png') 
bird_img = pygame.image.load('bird.png') 
pipe_img = pygame.image.load('pipe.png')

b1 = pygame.image.load('f_u.png') 
b2 = pygame.image.load('f_d.png') 
screen_size = (400,600) 
bg = pygame.transform.scale(bg, screen_size)
b1 = pygame.transform.scale(b1,(50,50)) 
b2 = pygame.transform.scale(b2,(50,50))


bird_img = pygame.transform.scale(bird_img,(50,50))
pipe_img = pygame.transform.scale(pipe_img,(70,600))
pipe_img_top = pygame.transform.flip(pipe_img, False, True)

pygame.mixer.init(frequency=22050, size=-16, channels=2, buffer=512)

flap_snd = pygame.mixer.Sound('flap.mp3')
point_snd = pygame.mixer.Sound('point.mp3') 
die_snd = pygame.mixer.Sound('die.mp3')

setting up screen 

In [3]:
pygame.init() 

screen = pygame.display.set_mode(screen_size) 
clock = pygame.time.Clock() 
run = True 
while run: 
    clock.tick(30) 
    for event in pygame.event.get(): 
        if event.type == pygame.QUIT: 
            run = False 

    screen.blit(bg,(0,0))
    
    pygame.display.update() 

pygame.quit() 


bird 

In [4]:
pygame.init() 
screen_size = (400,600) 
screen = pygame.display.set_mode(screen_size) 
class Bird: 
    def __init__(self): 
        self.x = 50 
        self.y = screen_size[1] // 2 # middle say 
        self.d = 0 
        self.images = [b1,b2] 
        self.curr = 0 
        self.anitime = 0 
        self.anispeed = 5 

    def draw(self): 
        # pygame.draw.rect(screen, (255,0,0), (self.x, self.y, 20,20))
        # screen.blit(bird_img, (self.x, self.y))
        screen.blit(self.images[self.curr], (self.x, self.y))
    def jump(self):
        self.d = -8# jump strength 
    
    def update(self): 
        self.d += 0.5 
        self.y += self.d

        self.anitime +=1 
        if self.anitime >= self.anispeed: 
            self.anitime = 0 
            self.curr = (self.curr +1) % len(self.images)

clock = pygame.time.Clock() 
bird = Bird() 
run = True 
while run: 
    clock.tick(30) 
    screen.fill((255,0,0))
    for event in pygame.event.get(): 
        if event.type == pygame.QUIT: 
            run = False
        if event.type == pygame.KEYDOWN: 
            if event.key == pygame.K_SPACE: 
                bird.jump() 
    
    bird.update() 
    bird.draw() 

    pygame.display.update() 

pygame.quit() 


pipe 

In [5]:
pygame.init() 
screen_size = (400,600) 
screen = pygame.display.set_mode(screen_size) 

class Pipe: 
    def __init__(self): 
        self.x = screen_size[0] # horizontal position of pipe on screen 
        self.h = np.random.randint(100,350) # height of top pieces use random for zigzagness
        self.passed = False

    def draw(self): 
        # pygame.draw.rect(screen, (0,255,0), (self.x, 0, 70, self.h)) 
        # pygame.draw.rect(screen, (0,255,0), (self.x, self.h + 150, 70, screen_size[1] - self.h - 150))
        screen.blit(pipe_img_top, (self.x, self.h - pipe_img_top.get_height()))
        screen.blit(pipe_img, (self.x, self.h + 120))        

    def update(self): 
        self.x += -5 # move to left after each frame 
    
    def collision(self,bird): 
        bird_rect = pygame.Rect(bird.x, bird.y, 20,20)
        pipe_rect_top = pygame.Rect(self.x, 0, 70, self.h-20) 
        pipe_rect_bottom = pygame.Rect(self.x, self.h + 120, 70, screen_size[1] - self.h-100)
        return bird_rect.colliderect(pipe_rect_top) or bird_rect.colliderect(pipe_rect_bottom)

clock = pygame.time.Clock() 
pipes = [Pipe()]
run = True 
while run: 
    clock.tick(30) 
    screen.fill((0,0,0))
    for event in pygame.event.get(): 
        if event.type == pygame.QUIT: 
            run = False

    if len(pipes) == 0 or pipes[-1].x < screen_size[0] - 200: 
        pipes.append(Pipe())

    for pipe in pipes:
        pipe.update() 
        pipe.draw() 

    pipes = [pipe for pipe in pipes if pipe.x+70 > 0]

    pygame.display.update() 

pygame.quit() 


game 

In [22]:
pygame.init() 
screen_size = (400,600) 
screen = pygame.display.set_mode(screen_size) 
clock = pygame.time.Clock() 

def main(): 
    bird = Bird()
    pipes = [Pipe()]
    score = 0 
    run = True 
    while run: 
        clock.tick(30)  #30
        # screen.fill((0,0,0))
        screen.blit(bg, (0,0)) # background 
        for event in pygame.event.get(): 
            if event.type == pygame.QUIT: 
                run = False

            if event.type == pygame.KEYDOWN: 
                if event.key == pygame.K_SPACE: 
                    bird.jump() 
                    flap_snd.play()
        
        bird.update() 
        bird.draw() 

        # new pipes add if needed
        if len(pipes) == 0 or pipes[-1].x < screen_size[0] - 200: 
            pipes.append(Pipe())

        # update and draw all pipes 
        for pipe in pipes:
            pipe.update() 
            pipe.draw() 
            if pipe.collision(bird):
                die_snd.play() 
                pygame.time.delay(1000)
                run = False 
            if pipe.x + 70 < bird.x and not pipe.passed: 
                point_snd.play()
                pipe.passed = True 
                score += 1 
        pipes = [pipe for pipe in pipes if pipe.x+70 > 0]  # removes off screen popes 

        if bird.y > screen_size[1] or bird.y < 0: # out of bounds 
            run = False 


        pygame.display.update() 


    pygame.quit() 
    print(f"Your score is: {score}")


main()


Your score is: 0
